In [1]:
import rmgpy
import numpy as np
from rmgpy.molecule.molecule import *
from rmgpy.species import *
from rmgpy.chemkin import *
from rmgpy.data.rmg import RMGDatabase
from IPython.display import display
from rmgpy.data.thermo import ThermoLibrary
from rmgpy.rmg.react import react
from rmgpy.species import Species
from rmgpy.reaction import Reaction
from rmgpy.data.rmg import get_db
from rmgpy.molecule.group import Group
from rmgpy.kinetics.arrhenius import ArrheniusBM
from rmgpy import settings
import time
import matplotlib.pyplot as plt
import matplotlib

In [2]:
settings

{'database.directory': '/home/khalil.nor/Code/RMG-database/input',
 'test_data.directory': '/home/khalil.nor/Code/RMG-Py/rmgpy/test_data'}

In [ ]:
#these families just had training rxns added to them 

#dict_keys(['1,2_Insertion_carbene', 'R_Addition_MultipleBond', 'XY_Addition_MultipleBond', 'Singlet_Carbene_Intra_Disproportionation', '1,2_Insertion_CO', '1+2_Cycloaddition', '1,3_Insertion_CO2', '1,3_sigmatropic_rearrangement'])

In [3]:
thermo_libs = [
'C1_C2_Fluorine', #adding Siddha's as first most trusted because this is the thermo library that Franklin used
'NCSU_C2_C8_PFAS', #adding Westmoreland's thermo as the second most trusted
'primaryThermoLibrary',
'Fluorine',
'FFCM1(-)',
'halogens',
'CHOF_G4',
'CHOCl_G4',
'CHOBr_G4',
'CHOFCl_G4',
'CHOFBr_G4',
'CHOFClBr_G4',
'DFT_QCI_thermo',
'2-BTP_G4',
'thermo_DFT_CCSDTF12_BAC',
'SulfurHaynes'
]

kin_families = ['R_Recombination', '1,2_Insertion_carbene', 'F_Abstraction', 'R_Addition_MultipleBond', 'XY_Addition_MultipleBond', 'Singlet_Carbene_Intra_Disproportionation', 'Intra_R_Add_Endocyclic', 'R_Addition_COm', '1,2_Insertion_CO', '1+2_Cycloaddition', '1,3_Insertion_CO2', '1,3_sigmatropic_rearrangement']


In [4]:
database = RMGDatabase()
database.load(
            path = settings['database.directory'],
            thermo_libraries = thermo_libs,
            transport_libraries = [],
            reaction_libraries = [],
            seed_mechanisms = [],#['BurkeH2O2inN2','ERC-FoundationFuelv0.9'],
            kinetics_families = kin_families,
            kinetics_depositories = ['training'],
            #frequenciesLibraries = self.statmechLibraries,
            depository = False, # Don't bother loading the depository information, as we don't use it
        )


In [6]:
database.kinetics.families

{'1+2_Cycloaddition': <ReactionFamily "1+2_Cycloaddition">,
 '1,2_Insertion_CO': <ReactionFamily "1,2_Insertion_CO">,
 '1,2_Insertion_carbene': <ReactionFamily "1,2_Insertion_carbene">,
 '1,3_Insertion_CO2': <ReactionFamily "1,3_Insertion_CO2">,
 '1,3_sigmatropic_rearrangement': <ReactionFamily "1,3_sigmatropic_rearrangement">,
 'F_Abstraction': <ReactionFamily "F_Abstraction">,
 'Intra_R_Add_Endocyclic': <ReactionFamily "Intra_R_Add_Endocyclic">,
 'R_Addition_COm': <ReactionFamily "R_Addition_COm">,
 'R_Addition_MultipleBond': <ReactionFamily "R_Addition_MultipleBond">,
 'R_Recombination': <ReactionFamily "R_Recombination">,
 'Singlet_Carbene_Intra_Disproportionation': <ReactionFamily "Singlet_Carbene_Intra_Disproportionation">,
 'XY_Addition_MultipleBond': <ReactionFamily "XY_Addition_MultipleBond">}

In [7]:
# 'Singlet_Carbene_Intra_Disproportionation' 
family_to_train = "Singlet_Carbene_Intra_Disproportionation"
family = database.kinetics.families[family_to_train]

In [8]:
family.clean_tree()

In [9]:
start = time.time()
family.generate_tree(thermo_database=database.thermo,
                     nprocs=1,
                     new_fraction_threshold_to_reopt_node=0.25,
                     max_batch_size=800,
                     extension_iter_max=2,
                     extension_iter_item_cap=100)

end = time.time()
print(end-start)

ERROR:root:10.0
ERROR:root:iter_max achieved terminating early
ERROR:root:iter_max achieved terminating early
ERROR:root:iter_max achieved terminating early


173.4738233089447


In [10]:
len(family.groups.entries)

17

In [11]:
family.groups.entries

{'CH_C_unsaturated': <Entry index=-1 label="CH_C_unsaturated">,
 'Root': <Entry index=0 label="Root">,
 'Root_2Br1sCl1sF1sH->H': <Entry index=1 label="Root_2Br1sCl1sF1sH->H">,
 'Root_N-2Br1sCl1sF1sH->H': <Entry index=2 label="Root_N-2Br1sCl1sF1sH->H">,
 'Root_2Br1sCl1sF1sH->H_Ext-3C-R_4R!H->Br': <Entry index=3 label="Root_2Br1sCl1sF1sH->H_Ext-3C-R_4R!H->Br">,
 'Root_2Br1sCl1sF1sH->H_Ext-3C-R_N-4R!H->Br': <Entry index=4 label="Root_2Br1sCl1sF1sH->H_Ext-3C-R_N-4R!H->Br">,
 'Root_N-2Br1sCl1sF1sH->H_Ext-3C-R_Ext-1C-R_4R!H->C': <Entry index=5 label="Root_N-2Br1sCl1sF1sH->H_Ext-3C-R_Ext-1C-R_4R!H->C">,
 'Root_N-2Br1sCl1sF1sH->H_Ext-3C-R_Ext-1C-R_N-4R!H->C': <Entry index=6 label="Root_N-2Br1sCl1sF1sH->H_Ext-3C-R_Ext-1C-R_N-4R!H->C">,
 'Root_2Br1sCl1sF1sH->H_Ext-3C-R_N-4R!H->Br_4CClFINOPSSi->F': <Entry index=7 label="Root_2Br1sCl1sF1sH->H_Ext-3C-R_N-4R!H->Br_4CClFINOPSSi->F">,
 'Root_2Br1sCl1sF1sH->H_Ext-3C-R_N-4R!H->Br_N-4CClFINOPSSi->F': <Entry index=8 label="Root_2Br1sCl1sF1sH->H_Ext-3C-R_N

In [12]:
start = time.time()
family.check_tree()
end = time.time()
print(end-start)

0.0009047985076904297


In [13]:
start = time.time()
family.regularize(thermo_database=database.thermo)
end = time.time()
print(end-start)

0.2708463668823242


In [14]:
start = time.time()
templateRxnMap = family.get_reaction_matches(thermo_database=database.thermo,remove_degeneracy=True,
                                             get_reverse=True,exact_matches_only=False,fix_labels=True)
end = time.time()
print(end-start)

0.27899622917175293


In [15]:
len(templateRxnMap)

17

In [16]:
family.clean_tree_rules()

In [17]:
start = time.time()
family.make_bm_rules_from_template_rxn_map(templateRxnMap)#,nprocs=6)
end = time.time()
print(end-start)

/home/khalil.nor/Code/RMG-Py/rmgpy/data/kinetics/family.py:3639: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  for rxns, label in rxnlists])
/work/westgroup/nora/miniconda/envs/rmg_env/lib/python3.7/site-packages/scipy/optimize/minpack.py:834: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
/work/westgroup/nora/miniconda/envs/rmg_env/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/work/westgroup/nora/miniconda/envs/rmg_env/lib/python3.7/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


0.3261549472808838


In [18]:
start = time.time()
family.check_tree()
end = time.time()
print(end-start)

0.0031697750091552734


In [19]:
start = time.time()
errors,uncertainties = family.cross_validate(iters=0,random_state=5,folds=0,ascend=False)
end = time.time()
print(end-start)

0.37302660942077637


In [20]:
save_path = os.path.join(settings['database.directory'], 'kinetics', 'families', family.name)
print(save_path)

/home/khalil.nor/Code/RMG-database/input/kinetics/families/Singlet_Carbene_Intra_Disproportionation


In [21]:
family.save(save_path)

In [ ]:
#ran to completion on May 7th, 6:48 pm